# SL Fidelidad - 01_Carga de datos

Fuentes disponibles:
- caracteristicas
- abonos
- eventos

In [18]:
# !pip install openpyxl
# !pip install xlrd

In [79]:
import pandas as pd
import numpy as np
from pathlib import Path
import pickle

import xlrd
from openpyxl import load_workbook

In [4]:
# seteo carpetas
BASE = Path().absolute().parent
RAW = Path(BASE/'raw')
DATA = Path(BASE/'data')

### Carga de datos y preparacion de tablas

#### Tablon de socios

In [20]:
# caracteristicas
df_soc = pd.read_excel(RAW/'socios_20200101.xlsx', engine='openpyxl')
df_soc.head()

,Socio,Nombre,Nro. Doc.,Sexo,F. Ingreso,Rec/Desc,Forma de pago,Tarjeta,Tipo de socio,Categoria,Estado,UCP
0,1,(N) COLOM MIGUEL,10607485,M,1952-06-27,<Ninguno>,Efectivo,<Ninguna>,VITALICIO,VITALICIO,Activo,NaT
1,2,TOMAS BERNARDI,1987009,M,1953-03-06,<Ninguno>,Efectivo,<Ninguna>,VITALICIO,VITALICIO,Activo,NaT
2,3,(N) FERREIRO MODESTO,452207,M,1953-03-06,<Ninguno>,Efectivo,<Ninguna>,VITALICIO,VITALICIO,Activo,NaT
3,4,(N) PONCELIZ HORACIO M,1004835,M,1954-04-02,<Ninguno>,Efectivo,<Ninguna>,VITALICIO,VITALICIO,Activo,NaT
4,5,(N) RIZZO CARMELO,2392330,M,1954-04-02,<Ninguno>,Efectivo,<Ninguna>,VITALICIO,VITALICIO,Activo,NaT


In [238]:
df_soc['Tipo de socio'].value_counts()

1 - SIMPLE             38716
2 - PLENO              12072
3 - INTERIOR           10689
VITALICIO               4078
5 - PATRIMONIAL          762
4 - EXTERIOR             329
HONORARIO                 31
HONORARIO SIMPLE           6
HONORARIO INTERIOR         2
HONORARIO VITALICIO        1
Name: Tipo de socio, dtype: int64

In [246]:
# tipo de socio
df_soc['tipo_socio'] = 'SIMPLE'
df_soc.loc[(df_soc['Tipo de socio'] == '3 - INTERIOR'), 'tipo_socio'] = 'INTERIOR'
df_soc.loc[(df_soc['Tipo de socio'] == '4 - EXTERIOR'), 'tipo_socio'] = 'EXTERIOR'
df_soc.loc[(df_soc['Tipo de socio'] == '5 - PATRIMONIAL'), 'tipo_socio'] = 'PATRIMONIAL'
df_soc.loc[(df_soc['Tipo de socio'] == '2 - PLENO'), 'tipo_socio'] = 'PLENO'
df_soc.loc[(df_soc['Tipo de socio'] == 'VITALICIO'), 'tipo_socio'] = 'VITALICIO'
df_soc['tipo_socio'].value_counts()

SIMPLE         38756
PLENO          12072
INTERIOR       10689
VITALICIO       4078
PATRIMONIAL      762
EXTERIOR         329
Name: tipo_socio, dtype: int64

In [251]:
# forma de pago
print(df_soc['Forma de pago'].value_counts())
df_soc.loc[(df_soc['Forma de pago'] == 'Débito Automático'), 'forma_pago'] = 'DEBITO'
df_soc.loc[(df_soc['Forma de pago'] == 'Efectivo'), 'forma_pago'] = 'EFECTIVO'

Débito Automático    48705
Efectivo             17981
Name: Forma de pago, dtype: int64


In [318]:
df_socios = df_soc[['Socio', 'Sexo', 'tipo_socio', 'forma_pago']]
df_socios = df_socios.set_index('Socio')
df_socios.head()

,Sexo,tipo_socio,forma_pago
Socio,,,
1,M,VITALICIO,EFECTIVO
2,M,VITALICIO,EFECTIVO
3,M,VITALICIO,EFECTIVO
4,M,VITALICIO,EFECTIVO
5,M,VITALICIO,EFECTIVO


#### Abonados

In [24]:
# abonos
df_abo = pd.read_csv(RAW/'abonos_historico.csv', sep=';', encoding = "ISO-8859-1")
df_abo.head()

,Socio,Abono,Apellido y Nombre,Unnamed: 3,UCP,Fila,Asiento,Sector,Seccion,Tarifa,Vencimiento,Mail,Telefono,Celular,Unnamed: 14,Original,Pago
0,NaN,381.0,ALVAREZ ROBERTO,NaN,NaN,1.0,10.0,ESTACIONAMIENTO,EST AV PERITO MORENO 2,ABONO E RENOVACION -5% Refund,31/12/2020,NaN,NaN,NaN,NaN,"$ 49.590,00","$ 49.590,00"
1,NaN,58074.0,ARRECEYGOR CARLOS HORACIO,NaN,NaN,1.0,7.0,ESTACIONAMIENTO,EST AV PERITO MORENO 2,ABONO E RENOVACION,31/12/2019,NaN,NaN,NaN,NaN,"$ 51.660,00","$ 51.660,00"
2,NaN,61704.0,BELLUOMO JUAN PABLO,NaN,NaN,1.0,39.0,ESTACIONAMIENTO,EST AV PERITO MORENO 2,ABONO E NUEVO,31/12/2019,NaN,NaN,NaN,NaN,"$ 24.000,00","$ 24.000,00"
3,NaN,4187.0,BERNASCHINA HECTOR FRANCISCO,NaN,NaN,1.0,16.0,ESTACIONAMIENTO,EST AV PERITO MORENO 2,ABONO E RENOVACION,31/12/2019,NaN,NaN,NaN,NaN,"$ 41.250,00","$ 41.250,00"
4,NaN,32918.0,BOUZON GUSTAVO BALTASAR,NaN,NaN,1.0,1.0,ESTACIONAMIENTO,EST AV PERITO MORENO 2,ABONO AFA ANUAL,31/12/2020,NaN,NaN,NaN,NaN,"$ 0,00","$ 0,00"


In [264]:
df_abo = df_abo[df_abo['Sector']!='ESTACIONAMIENTO']
df_abo['Sector'].value_counts()

PLATEA SUR       2903
PLATEA NORTE     2548
POLIDEPORTIVO     652
Name: Sector, dtype: int64

In [266]:
df_abo['abo_year'] = df_abo['Vencimiento'].str.slice(start = -4)
df_abo['abo_year'].value_counts()

2020    3527
2019    2366
2023     118
2021      52
2022      20
2024      16
2029       2
2026       2
Name: abo_year, dtype: int64

In [306]:
df_abo.shape

(6104, 18)

In [314]:
len(df_abo['Socio'].unique())

4958

In [303]:
# genero dataframe con variables por abono
df_abonos = pd.concat([
    # bidegain 2020
    pd.DataFrame(
        {'socio':df_abo.loc[(df_abo['abo_year'] == '2020') & (df_abo['Sector'].isin(['PLATEA SUR', 'PLATEA NORTE']))\
                              , 'Socio']
         ,'abo_bid_2020':1}
    ).set_index('socio'),
    # bidegain 2021
    pd.DataFrame(
        {'socio':df_abo.loc[(df_abo['abo_year'] == '2021') & (df_abo['Sector'].isin(['PLATEA SUR', 'PLATEA NORTE']))\
                              , 'Socio']
         ,'abo_bid_2021':1}
    ).set_index('socio'),
    # polideportivo 2020
    pd.DataFrame(
        {'socio':df_abo.loc[(df_abo['abo_year'] == '2020') & (df_abo['Sector']=='POLIDEPORTIVO'), 'Socio']
         ,'abo_pol_2020':1}
    ).set_index('socio'),
    # polideportivo 2021
    pd.DataFrame(
        {'socio':df_abo.loc[(df_abo['abo_year'] == '2021') & (df_abo['Sector']=='POLIDEPORTIVO'), 'Socio']
         ,'abo_pol_2021':1}
    ).set_index('socio')
    ], axis=1)

In [305]:
df_abonos.head()

,abo_bid_2020,abo_bid_2021,abo_pol_2020,abo_pol_2021,aux
socio,,,,,
1475.0,1.0,NaN,NaN,NaN,NaN
1630.0,1.0,NaN,NaN,NaN,NaN
1771.0,1.0,NaN,NaN,NaN,NaN
1930.0,1.0,NaN,NaN,NaN,NaN
1941.0,1.0,NaN,NaN,NaN,NaN


In [304]:
df_abonos['aux'] = df_abonos['abo_bid_2020']+df_abonos['abo_bid_2021']+df_abonos['abo_pol_2020']+df_abonos['abo_pol_2021']
df_abonos['aux'].value_counts()

Series([], Name: aux, dtype: int64)

In [85]:
# eventos
eve = load_workbook(filename = RAW/'eventos.xlsx')
eve_sheets = eve.sheetnames

In [111]:
print(eve_sheets)
eve_sheets.remove(['Abonado', 'UNOxSOCIO'])

['154', '155', '156', '157', '158', '159', '160', '162', '166', '167', '170', '171', '172', '173', '174', '175', '176', '179', '187', '188', '189', '190', '192', '193', '194', '195', '199', '200', '201']


In [104]:
def info_evento(sheetname):
    df_eve = pd.read_excel(RAW/'eventos.xlsx', engine='openpyxl', sheet_name=sheetname)
    dic_eve = {
        "evento": df_eve['Evento'][0], 
        "fecha": df_eve['Fecha'][0],
        "socios": df_eve.loc[df_eve['Tipo'] == 'Socio']['Numero'].unique(),
        "publico": len(set([str(x) for x in df_eve[['Tipo', 'Numero']].values]))
        }
    return dic_eve

In [103]:
# ejemplo
info_evento(eve_sheets[1])

{'sheetname': '154',
 'evento': '154 - Lanús - 2018-08-19',
 'fecha': Timestamp('2018-08-19 18:48:00'),
 'socios': array([ 21065,  78406, 105137, ...,  34832,  10406,  46192], dtype=int64),
 'publico': 16950}

In [120]:
eventos = {}
for eve in range(len(eve_sheets)):
    eventos[eve] = info_evento(eve_sheets[eve])

In [128]:
print(len(eventos))
eventos[0]

29


{'evento': '154 - Lanús - 2018-08-19',
 'fecha': Timestamp('2018-08-19 18:48:00'),
 'socios': array([ 21065,  78406, 105137, ...,  34832,  10406,  46192], dtype=int64),
 'publico': 16950}

In [143]:
# publico promedio
np.mean(np.array([int(eventos[x]['publico']) for x in eventos]))

In [158]:
lista = list(eventos[0]['socios'])
print(len(lista))
lista.extend(list(eventos[1]['socios']))
print(len(lista))

11242
17344


In [165]:
# socios totales que asistieron a algun evento
socios_eve = []
for eve in eventos:
    socios_eve.extend(list(eventos[eve]['socios']))
print(len(socios_eve))
socios_eve = list(set(socios_eve))
print(len(socios_eve))

336058
47585


In [227]:
socios_eventos = pd.DataFrame({'socio': socios_eve
                              ,'eve_2018': np.zeros(len(socios_eve))
                              ,'eve_2019': np.zeros(len(socios_eve))
                              ,'eve_2020': np.zeros(len(socios_eve))})

In [223]:
for evento in eventos:
    if eventos[evento]['fecha'].year == 2018:
        socios_eventos['eve_2018'][0] += (socios_eventos['socio'][0] in eventos[evento]['socios'])
    if eventos[evento]['fecha'].year == 2019:
        socios_eventos['eve_2019'][0] += (socios_eventos['socio'][0] in eventos[evento]['socios'])    

In [224]:
socios_eventos.head()

,socio,eve_2018,eve_2019,eve_2020
0,258.0,2.0,5.0,0.0
1,285.0,0.0,0.0,0.0
2,425.0,0.0,0.0,0.0
3,431.0,0.0,0.0,0.0
4,449.0,0.0,0.0,0.0


In [228]:
# cantidad de partidos asistidos por socio por año
for socio in range(len(socios_eventos['socio'])):
    for evento in eventos:
        if eventos[evento]['fecha'].year == 2018:
            socios_eventos['eve_2018'][socio] += (socios_eventos['socio'][socio] in eventos[evento]['socios'])
        if eventos[evento]['fecha'].year == 2019:
            socios_eventos['eve_2019'][socio] += (socios_eventos['socio'][socio] in eventos[evento]['socios'])  
        if eventos[evento]['fecha'].year == 2020:
            socios_eventos['eve_2020'][socio] += (socios_eventos['socio'][socio] in eventos[evento]['socios'])

In [319]:
df_eventos = socios_eventos.set_index('socio')
df_eventos.head()

,eve_2018,eve_2019,eve_2020
socio,,,
258.0,2.0,5.0,0.0
285.0,1.0,6.0,0.0
425.0,0.0,0.0,0.0
431.0,2.0,6.0,0.0
449.0,4.0,15.0,0.0


### Junto las bases

- Socios: df_soc
- Abonos: df_abo
- Eventos: df_eve

In [237]:
#
print(df_soc.dtypes)
print(df_abo.dtypes)

Socio                     int64
Nombre                   object
Nro. Doc.                object
Sexo                     object
F. Ingreso       datetime64[ns]
Rec/Desc                 object
Forma de pago            object
Tarjeta                  object
Tipo de socio            object
Categoria                object
Estado                   object
UCP              datetime64[ns]
dtype: object
Socio                float64
Abono                float64
Apellido y Nombre     object
Unnamed: 3           float64
UCP                   object
Fila                 float64
Asiento              float64
Sector                object
Seccion               object
Tarifa                object
Vencimiento           object
Mail                  object
Telefono              object
Celular               object
Unnamed: 14          float64
Original              object
Pago                  object
dtype: object
